<a href="https://colab.research.google.com/github/Estebanarci2/Marketing-analitica3/blob/main/B)EXPLORATORIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import sqlite3 as sql
#import sklearn.prepocessing
import joblib
from ipywidgets import interact   ## analisis interactivo
#from sklearn import neighborns ## basado en contenido un solo

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## conectar la base de datos

conn = sql.connect ("/content/drive/MyDrive/analitica3/marketing /db_movies")
cur=conn.cursor()

In [4]:
# consultar bases de datos que se pueden consultar por medio de sql
cur.execute("Select name from sqlite_master where type='table'")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('usuarios_fil',),
 ('movies_fil',)]

In [7]:
consulta_sql = """
     SELECT SUBSTR(genres, 1, INSTR(genres, '|') - 1) AS genero, COUNT(*) AS total_peliculas
     FROM movies
     GROUP BY genero
     ORDER BY total_peliculas DESC
"""

In [8]:
pd.read_sql_query(consulta_sql, conn)

,genero,total_peliculas
0,,2851
1,Comedy,1833
2,Action,1768
3,Drama,1173
4,Adventure,641
5,Crime,525
6,Horror,301
7,Animation,268
8,Children,193
9,Documentary,47
